In [1]:
import pandas as pd
import re 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fnmatch
from yelpapi import YelpAPI
import json
from decimal import *

%matplotlib inline

In [2]:
pov_data = pd.read_excel('../Data/poverty_rates.xlsx', sheet_name = 'Percents', header = 2)
pov_data

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,0.026037,0.012483,0.045000,0.004918,0.034508,0.011601,0.034091,0.008723,...,0.036052,0.009398,0.024596,0.005037,0.036945,0.007138,0.012765,0.003430,0.028517,0.009377
1,.50 to .74,poverty,0.024342,0.012483,0.023456,0.003069,0.022094,0.010641,0.016845,0.005842,...,0.019741,0.007608,0.021070,0.004877,0.014934,0.004207,0.010619,0.003978,0.011689,0.004958
2,.75 to .99,poverty,0.022202,0.008382,0.037121,0.003978,0.026306,0.010788,0.023047,0.007602,...,0.034112,0.011686,0.020271,0.004767,0.024564,0.005882,0.008348,0.003414,0.015594,0.005010
3,1.00 to 1.24,low-income,0.030584,0.012305,0.031564,0.003722,0.026528,0.010641,0.041693,0.013684,...,0.021880,0.006415,0.024117,0.004423,0.023726,0.005643,0.021285,0.006578,0.020476,0.006423
4,1.25 to 1.49,low-income,0.033081,0.010700,0.039468,0.004527,0.027562,0.009163,0.030210,0.010083,...,0.039582,0.011885,0.034043,0.007765,0.029409,0.006859,0.010087,0.002945,0.023045,0.006860
5,1.50 to 1.74,low-income,0.038698,0.012037,0.038541,0.004502,0.034360,0.010345,0.048496,0.013404,...,0.032074,0.009597,0.026598,0.005283,0.027834,0.005523,0.012201,0.003759,0.025152,0.007039
6,1.75 to 1.84,low-income,0.019082,0.012127,0.016473,0.002711,0.020838,0.011158,0.018326,0.008083,...,0.013526,0.007260,0.014006,0.004374,0.010906,0.003370,0.008896,0.002631,0.009223,0.003468
7,1.85 to 1.99,low-income,0.029157,0.010878,0.020719,0.003376,0.031848,0.011158,0.025768,0.008283,...,0.021929,0.006663,0.024473,0.004755,0.020656,0.005762,0.012358,0.006061,0.013976,0.003879
8,2.00 to 2.99,NaN,0.169327,0.021400,0.146650,0.007731,0.202172,0.029262,0.197423,0.023648,...,0.179712,0.022576,0.178449,0.014190,0.147064,0.012102,0.086676,0.009163,0.156998,0.018035
9,3.00 to 3.99,NaN,0.192867,0.024253,0.134462,0.007917,0.196483,0.031848,0.165773,0.021407,...,0.167330,0.022327,0.161986,0.009325,0.172346,0.014156,0.099018,0.010431,0.156073,0.015004


In [3]:
pov_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rate                   13 non-null     object 
 1   Income level           8 non-null      object 
 2   Cheatham County, TN    13 non-null     float64
 3   Error                  13 non-null     float64
 4   Davidson County, TN    13 non-null     float64
 5   Error.1                13 non-null     float64
 6   Dickson County, TN     13 non-null     float64
 7   Error.2                13 non-null     float64
 8   Maury County, TN       13 non-null     float64
 9   Error.3                13 non-null     float64
 10  Montgomery County, TN  13 non-null     float64
 11  Error.4                13 non-null     float64
 12  Robertson County, TN   13 non-null     float64
 13  Error.5                13 non-null     float64
 14  Rutherford County, TN  13 non-null     float64
 15  Error.6 

In [4]:
pov_data[['Cheatham County, TN', 'Error', 'Davidson County, TN', 'Error.1', 'Dickson County, TN', 'Error.2', 'Maury County, TN', 'Error.3', 'Montgomery County, TN', 'Error.4', 'Robertson County, TN', 'Error.5', 'Rutherford County, TN', 'Error.6', 'Sumner County, TN', 'Error.7', 'Williamson County, TN', 'Error.8', 'Wilson County, TN', 'Error.9']] = pov_data[['Cheatham County, TN', 'Error', 'Davidson County, TN', 'Error.1', 'Dickson County, TN', 'Error.2', 'Maury County, TN', 'Error.3', 'Montgomery County, TN', 'Error.4', 'Robertson County, TN', 'Error.5', 'Rutherford County, TN', 'Error.6', 'Sumner County, TN', 'Error.7', 'Williamson County, TN', 'Error.8', 'Wilson County, TN', 'Error.9']].apply(lambda x: x*100)
pov_data

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,2.603656,1.248328,4.499965,0.491754,3.450824,1.160127,3.409091,0.872279,...,3.605172,0.939831,2.459580,0.503710,3.694547,0.713787,1.276489,0.343008,2.851711,0.937725
1,.50 to .74,poverty,2.434240,1.248328,2.345583,0.306947,2.209414,1.064066,1.684539,0.584187,...,1.974142,0.760816,2.106983,0.487739,1.493371,0.420696,1.061914,0.397826,1.168945,0.495838
2,.75 to .99,poverty,2.220241,0.838163,3.712135,0.397752,2.630607,1.078844,2.304738,0.760243,...,3.411238,1.168573,2.027127,0.476682,2.456385,0.588177,0.834808,0.341441,1.559449,0.500976
3,1.00 to 1.24,low-income,3.058404,1.230495,3.156434,0.372173,2.652775,1.064066,4.169334,1.368438,...,2.187966,0.641472,2.411666,0.442282,2.372645,0.564251,2.128526,0.657823,2.047580,0.642277
4,1.25 to 1.49,low-income,3.308070,1.069996,3.946822,0.452746,2.756226,0.916279,3.020967,1.008323,...,3.958230,1.188463,3.404344,0.776451,2.940883,0.685874,1.008661,0.294454,2.304491,0.685952
5,1.50 to 1.74,low-income,3.869817,1.203745,3.854098,0.450188,3.436045,1.034508,4.849552,1.340429,...,3.207360,0.959722,2.659836,0.528281,2.783372,0.552288,1.220104,0.375899,2.515158,0.703936
6,1.75 to 1.84,low-income,1.908159,1.212662,1.647280,0.271136,2.083795,1.115791,1.832586,0.808259,...,1.352561,0.726007,1.400560,0.437368,1.090619,0.336955,0.889627,0.263129,0.922310,0.346830
7,1.85 to 1.99,low-income,2.915738,1.087829,2.071889,0.337641,3.184808,1.115791,2.576825,0.828265,...,2.192939,0.666335,2.447295,0.475453,2.065597,0.576214,1.235767,0.606137,1.397595,0.387935
8,2.00 to 2.99,NaN,16.932679,2.139991,14.665013,0.773122,20.217247,2.926180,19.742318,2.364757,...,17.971159,2.257583,17.844857,1.418989,14.706410,1.210248,8.667596,0.916253,15.699825,1.803515
9,3.00 to 3.99,NaN,19.286670,2.425323,13.446179,0.791666,19.648267,3.184808,16.577305,2.140685,...,16.732969,2.232720,16.198585,0.932478,17.234573,1.415612,9.901796,1.043119,15.607337,1.500360


In [5]:
pov_data.round(2)

,Rate,Income level,"Cheatham County, TN",Error,"Davidson County, TN",Error.1,"Dickson County, TN",Error.2,"Maury County, TN",Error.3,...,"Robertson County, TN",Error.5,"Rutherford County, TN",Error.6,"Sumner County, TN",Error.7,"Williamson County, TN",Error.8,"Wilson County, TN",Error.9
0,Under .50,poverty,2.60,1.25,4.50,0.49,3.45,1.16,3.41,0.87,...,3.61,0.94,2.46,0.50,3.69,0.71,1.28,0.34,2.85,0.94
1,.50 to .74,poverty,2.43,1.25,2.35,0.31,2.21,1.06,1.68,0.58,...,1.97,0.76,2.11,0.49,1.49,0.42,1.06,0.40,1.17,0.50
2,.75 to .99,poverty,2.22,0.84,3.71,0.40,2.63,1.08,2.30,0.76,...,3.41,1.17,2.03,0.48,2.46,0.59,0.83,0.34,1.56,0.50
3,1.00 to 1.24,low-income,3.06,1.23,3.16,0.37,2.65,1.06,4.17,1.37,...,2.19,0.64,2.41,0.44,2.37,0.56,2.13,0.66,2.05,0.64
4,1.25 to 1.49,low-income,3.31,1.07,3.95,0.45,2.76,0.92,3.02,1.01,...,3.96,1.19,3.40,0.78,2.94,0.69,1.01,0.29,2.30,0.69
5,1.50 to 1.74,low-income,3.87,1.20,3.85,0.45,3.44,1.03,4.85,1.34,...,3.21,0.96,2.66,0.53,2.78,0.55,1.22,0.38,2.52,0.70
6,1.75 to 1.84,low-income,1.91,1.21,1.65,0.27,2.08,1.12,1.83,0.81,...,1.35,0.73,1.40,0.44,1.09,0.34,0.89,0.26,0.92,0.35
7,1.85 to 1.99,low-income,2.92,1.09,2.07,0.34,3.18,1.12,2.58,0.83,...,2.19,0.67,2.45,0.48,2.07,0.58,1.24,0.61,1.40,0.39
8,2.00 to 2.99,NaN,16.93,2.14,14.67,0.77,20.22,2.93,19.74,2.36,...,17.97,2.26,17.84,1.42,14.71,1.21,8.67,0.92,15.70,1.80
9,3.00 to 3.99,NaN,19.29,2.43,13.45,0.79,19.65,3.18,16.58,2.14,...,16.73,2.23,16.20,0.93,17.23,1.42,9.90,1.04,15.61,1.50


In [6]:
with open('keys.json') as fi:
    credentials = json.load(fi)

In [7]:
api_key = credentials['yelp_key']
client_id = credentials['yelp_id']

In [8]:
yelp_api = YelpAPI(api_key)
response = yelp_api.search_query(term = 'park', location = 'Wilson County, TN', search_limit = 200)
response

{'businesses': [{'id': 'R31Aoy1vHCUiSbHYi1rcIw',
   'alias': 'don-fox-park-lebanon',
   'name': 'Don Fox Park',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/yFx37Us-AVaifZQ1F3idiw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/don-fox-park-lebanon?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 11,
   'categories': [{'alias': 'parks', 'title': 'Parks'}],
   'rating': 3.5,
   'coordinates': {'latitude': 36.2182684, 'longitude': -86.3116517},
   'transactions': [],
   'location': {'address1': '416 Baird Park Cir',
    'address2': '',
    'address3': '',
    'city': 'Lebanon',
    'zip_code': '37087',
    'country': 'US',
    'state': 'TN',
    'display_address': ['416 Baird Park Cir', 'Lebanon, TN 37087']},
   'phone': '+16154490303',
   'display_phone': '(615) 449-0303',
   'distance': 7064.107658925387},
  {'id': '6du6r19vgQ2eRmXb3r4M_g',
   'alias': 

In [9]:
cols = list(response['businesses'][0].keys())
data = pd.DataFrame(columns=cols)

for biz in response['businesses']:
    data = data.append(biz, ignore_index=True)
                    
data.head(50)   #has 20 parks no matter limit is 100 or 200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance
0,R31Aoy1vHCUiSbHYi1rcIw,don-fox-park-lebanon,Don Fox Park,https://s3-media4.fl.yelpcdn.com/bphoto/yFx37U...,False,https://www.yelp.com/biz/don-fox-park-lebanon?...,11,"[{'alias': 'parks', 'title': 'Parks'}]",3.5,"{'latitude': 36.2182684, 'longitude': -86.3116...",[],"{'address1': '416 Baird Park Cir', 'address2':...",+16154490303,(615) 449-0303,7064.107659
1,6du6r19vgQ2eRmXb3r4M_g,cedars-of-lebanon-state-park-lebanon-4,Cedars Of Lebanon State Park,https://s3-media4.fl.yelpcdn.com/bphoto/z2txvE...,False,https://www.yelp.com/biz/cedars-of-lebanon-sta...,41,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.091612, 'longitude': -86.330965}",[],"{'address1': '328 Cedar Forest Rd', 'address2'...",+16154432769,(615) 443-2769,7391.371636
2,EbxNiXeQMGM84Gl5KDLahA,rock-island-state-park-rock-island-4,Rock Island State Park,https://s3-media3.fl.yelpcdn.com/bphoto/t3LetC...,False,https://www.yelp.com/biz/rock-island-state-par...,57,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.5,"{'latitude': 35.8100143074989, 'longitude': -8...",[],"{'address1': '82 Beach Rd', 'address2': None, ...",+19318374770,(931) 837-4770,70966.759737
3,DiG2uoaL8ZLQcpGlkHraag,long-hunter-state-park-hermitage,Long Hunter State Park,https://s3-media1.fl.yelpcdn.com/bphoto/KLoy5s...,False,https://www.yelp.com/biz/long-hunter-state-par...,38,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.0943884, 'longitude': -86.5571...",[],"{'address1': '2910 Hobson Pike', 'address2': '...",+16158852422,(615) 885-2422,23605.899650
4,Q9wU-l_5R9AbhbAGOujatw,sellars-state-farm-archaeological-area-lebanon,Sellars State Farm Archaeological Area,https://s3-media1.fl.yelpcdn.com/bphoto/8_5pAY...,False,https://www.yelp.com/biz/sellars-state-farm-ar...,1,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.0,"{'latitude': 36.1654368105194, 'longitude': -8...",[],"{'address1': '433 Poplar Hill Rd', 'address2':...",,,5778.970145
5,tigsP5nwiJV8KmvC6OQ2_w,barfield-crescent-park-murfreesboro,Barfield Crescent Park,https://s3-media2.fl.yelpcdn.com/bphoto/aTmm7W...,False,https://www.yelp.com/biz/barfield-crescent-par...,24,"[{'alias': 'discgolf', 'title': 'Disc Golf'}, ...",5.0,"{'latitude': 35.7862282, 'longitude': -86.416069}",[],"{'address1': '697 Barfield Crescent Rd', 'addr...",+16158905333,(615) 890-5333,42178.817689
6,njenDpIerIT38E4lgIhjkA,gregory-mill-dam-park-smyrna,Gregory Mill Dam Park,https://s3-media2.fl.yelpcdn.com/bphoto/4v2vjK...,False,https://www.yelp.com/biz/gregory-mill-dam-park...,4,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 35.9739189, 'longitude': -86.5039...",[],"{'address1': '390 Enon Springs Rd E', 'address...",+16154599773,(615) 459-9773,26936.111173
7,nRbgiD7uTBqfV_yS8uVfAw,seven-points-recreation-area-hermitage,Seven Points Recreation Area,https://s3-media2.fl.yelpcdn.com/bphoto/zlVu0v...,False,https://www.yelp.com/biz/seven-points-recreati...,9,"[{'alias': 'boating', 'title': 'Boating'}, {'a...",4.5,"{'latitude': 36.1338654, 'longitude': -86.5704...",[],"{'address1': '1810 Stewarts Ferry Pike', 'addr...",+16158895198,(615) 889-5198,23926.939422
8,emWQlqb8Lnr9HKNNXP7jsQ,bledsoe-creek-state-park-gallatin-2,Bledsoe Creek State Park,https://s3-media3.fl.yelpcdn.com/bphoto/_uNcGk...,False,https://www.yelp.com/biz/bledsoe-creek-state-p...,18,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.379714, 'longitude': -86.356685}",[],"{'address1': '400 Zieglers Fort Rd', 'address2...",+16154523706,(615) 452-3706,25408.900958
9,wZtPSFKyUhLQNy02cuRPSQ,hermitage-park-hermitage,Hermitage Park,https://s3-media2.fl.yelpcdn.com/bphoto/bmZw7f...,False,https://www.yelp.com/biz/hermitage-park-hermit...,4,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.1747932434082, 'longitude': -8...",[],"{'address1': '4000 Dodson Chapel Rd', 'address...",,,27772.10032

In [13]:
yelp_api = YelpAPI(api_key)
response2 = yelp_api.search_query(term = 'park', location = 'Dickson County, TN', search_limit = 300)
response2

{'businesses': [{'id': '8ypQ14bfksJtA7hL2nEmcw',
   'alias': 'montgomery-bell-state-park-burns',
   'name': 'Montgomery Bell State Park',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/FBF26ob55ak9ag4e8-WsUQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/montgomery-bell-state-park-burns?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 46,
   'categories': [{'alias': 'parks', 'title': 'Parks'},
    {'alias': 'hiking', 'title': 'Hiking'},
    {'alias': 'campgrounds', 'title': 'Campgrounds'}],
   'rating': 4.0,
   'coordinates': {'latitude': 36.095961, 'longitude': -87.286169},
   'transactions': [],
   'location': {'address1': '1020 Jackson Hill Rd',
    'address2': '',
    'address3': '',
    'city': 'Burns',
    'zip_code': '37029',
    'country': 'US',
    'state': 'TN',
    'display_address': ['1020 Jackson Hill Rd', 'Burns, TN 37029']},
   'phone': '+

In [14]:
cols = list(response2['businesses'][0].keys())
data2 = pd.DataFrame(columns=cols)

for biz in response2['businesses']:
    data2 = data2.append(biz, ignore_index=True)
                    
data2.head(50)   #has 20 parks no matter limit is 100 or 200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance
0,8ypQ14bfksJtA7hL2nEmcw,montgomery-bell-state-park-burns,Montgomery Bell State Park,https://s3-media4.fl.yelpcdn.com/bphoto/FBF26o...,False,https://www.yelp.com/biz/montgomery-bell-state...,46,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.095961, 'longitude': -87.286169}",[],"{'address1': '1020 Jackson Hill Rd', 'address2...",+16157979052,(615) 797-9052,8508.320560
1,G3FSsyHWzJ-s4ndaC-C95Q,henslee-park-dickson,Henslee Park,,False,https://www.yelp.com/biz/henslee-park-dickson?...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.087676, 'longitude': -87.397686}",[],"{'address1': '800 Hwy 70 W', 'address2': '', '...",,,7413.691761
2,RtlbW2Wgm0rVKKdv4uJq-w,buckner-park-swimming-pool-dickson,Buckner Park Swimming Pool,https://s3-media4.fl.yelpcdn.com/bphoto/2sYGck...,False,https://www.yelp.com/biz/buckner-park-swimming...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.090895, 'longitude': -87.409688}",[],"{'address1': '120 Upper Lake Dr', 'address2': ...",+16154411467,(615) 441-1467,7702.834890
3,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,21700.465403
4,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,29951.482462
5,aowWVmdNdlPcdajfpNKSMA,bowie-park-and-nature-center-fairview,Bowie Park & Nature Center,https://s3-media2.fl.yelpcdn.com/bphoto/nLQ8nE...,False,https://www.yelp.com/biz/bowie-park-and-nature...,7,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 35.969821, 'longitude': -87.134778}",[],"{'address1': '7211 Bowie Lake Rd', 'address2':...",+16157995544,(615) 799-5544,28032.706204
6,JfZLtsP7oir54iGZD2qyFQ,maury-county-park-columbia-2,Maury County Park,https://s3-media3.fl.yelpcdn.com/bphoto/yS60eJ...,False,https://www.yelp.com/biz/maury-county-park-col...,4,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 35.6111418560421, 'longitude': -8...",[],"{'address1': '705 Lion Pkwy', 'address2': '', ...",,,64635.501006
7,P7nobv_tPta4NCyC0mXrtw,oak-grove-war-memorial-walking-trail-park-oak-...,Oak Grove War Memorial Walking Trail Park,https://s3-media2.fl.yelpcdn.com/bphoto/AkvbUC...,False,https://www.yelp.com/biz/oak-grove-war-memoria...,5,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",5.0,"{'latitude': 36.6807358486363, 'longitude': -8...",[],"{'address1': '101 Walter Garrett Ln', 'address...",+12704395675,(270) 439-5675,59946.006688
8,kqWx1rjeuHh_gFf1o2eDxg,nathan-bedford-forrest-state-park-eva,Nathan Bedford Forrest State Park,https://s3-media1.fl.yelpcdn.com/bphoto/51GBBk...,False,https://www.yelp.com/biz/nathan-bedford-forres...,6,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.0833429, 'longitude': -87.9854...",[],"{'address1': '1825 Pilot Knob Rd', 'address2':...",+17315936445,(731) 593-6445,56808.925062
9,wZtPSFKyUhLQNy02cuRPSQ,hermitage-park-hermitage,Hermitage Park,https://s3-media2.fl.yelpcdn.com/bphoto/bmZw7f...,False,https://www.yelp.com/biz/hermitage-park-hermit...,4,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.1747932434082, 'longitude': -8...",[],"{'address1': '4000 Dodson Chapel Rd', 'address...",,,66828.942420


In [15]:
yelp_api = YelpAPI(api_key)
response3 = yelp_api.search_query(term = 'park', location = 'Davidson County, TN', search_limit = 200)
response3

{'businesses': [{'id': 'WjWDl782jABdLzUu9nof3g',
   'alias': 'shelby-park-nashville',
   'name': 'Shelby Park',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFPJbOHMB3doOgczNgA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/shelby-park-nashville?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 56,
   'categories': [{'alias': 'parks', 'title': 'Parks'},
    {'alias': 'playgrounds', 'title': 'Playgrounds'}],
   'rating': 4.5,
   'coordinates': {'latitude': 36.173435340130204,
    'longitude': -86.73402597753878},
   'transactions': [],
   'location': {'address1': '2021 Fatherland St',
    'address2': '',
    'address3': '',
    'city': 'Nashville',
    'zip_code': '37206',
    'country': 'US',
    'state': 'TN',
    'display_address': ['2021 Fatherland St', 'Nashville, TN 37206']},
   'phone': '+16158628474',
   'display_phone': '(615) 862-8474',
   '

In [16]:
cols = list(response3['businesses'][0].keys())
data3 = pd.DataFrame(columns=cols)

for biz in response3['businesses']:
    data3 = data3.append(biz, ignore_index=True)
                    
data3.head(50)   #has 20 parks no matter limit is 100 or 200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,4675.104705,NaN
1,yu9e-_S7U97kleJQw0eOzQ,richland-creek-greenway-nashville,Richland Creek Greenway,https://s3-media4.fl.yelpcdn.com/bphoto/fiqTzB...,False,https://www.yelp.com/biz/richland-creek-greenw...,30,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.5,"{'latitude': 36.140481933298005, 'longitude': ...",[],"{'address1': '4601 Murphy Rd', 'address2': '',...",,,7350.416823,NaN
2,Wjv13Wi_9RPRJPg-uMkcRQ,bicentennial-capitol-mall-state-park-nashville-2,Bicentennial Capitol Mall State Park,https://s3-media4.fl.yelpcdn.com/bphoto/I5sZVJ...,False,https://www.yelp.com/biz/bicentennial-capitol-...,86,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.1680549, 'longitude': -86.7863...",[],"{'address1': '600 James Robertson Pkwy', 'addr...",+16157415280,(615) 741-5280,2003.591226,NaN
3,CRmZDx9rW3HFo0Z9FbfyEw,cumberland-park-nashville,Cumberland Park,https://s3-media1.fl.yelpcdn.com/bphoto/xRQAmw...,False,https://www.yelp.com/biz/cumberland-park-nashv...,32,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.1626862381255, 'longitude': -8...",[],"{'address1': '592 S 1st St', 'address2': '', '...",,,2839.804616,NaN
4,73esU0u3hYIPPqCScAA2HQ,centennial-park-nashville,Centennial Park,https://s3-media3.fl.yelpcdn.com/bphoto/yiE3AL...,False,https://www.yelp.com/biz/centennial-park-nashv...,219,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.149064, 'longitude': -86.811993}",[],"{'address1': '2500 W End Ave', 'address2': '',...",,,4822.001912,NaN
5,zzXDi0Pdv0s84M-oQaIa_g,radnor-lake-state-park-nashville,Radnor Lake State Park,https://s3-media1.fl.yelpcdn.com/bphoto/x_k73A...,False,https://www.yelp.com/biz/radnor-lake-state-par...,208,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.062764, 'longitude': -86.809891}",[],"{'address1': '1160 Otter Creek Rd', 'address2'...",+16153733467,(615) 373-3467,13897.680563,NaN
6,N9abYpdjbGNOMv3adCqtDg,frankie-pierce-park-nashville,Frankie Pierce Park,https://s3-media2.fl.yelpcdn.com/bphoto/rn5rIn...,False,https://www.yelp.com/biz/frankie-pierce-park-n...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.1650388672316, 'longitude': -8...",[],"{'address1': '130 LifeWay Plz', 'address2': ''...",,,2385.544919,NaN
7,GoLUrmNHkh1x__YUUr21RQ,two-rivers-park-nashville,Two Rivers Park,https://s3-media1.fl.yelpcdn.com/bphoto/YMMGc5...,False,https://www.yelp.com/biz/two-rivers-park-nashv...,15,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.185593, 'longitude': -86.678432}",[],"{'address1': '3150 Mcgavock Pike', 'address2':...",+16158837781,(615) 883-7781,9452.182811,NaN
8,2-qIaZy7A6WWXSPfyX60eA,percy-warner-park-nashville,Percy Warner Park,https://s3-media2.fl.yelpcdn.com/bphoto/vMo9p2...,False,https://www.yelp.com/biz/percy-warner-park-nas...,116,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.5,"{'latitude': 36.055337, 'longitude': -86.890643}",[],"{'address1': '2500 Old Hickory Blvd', 'address...",+16158628415,(615) 862-8415,17410.325973,NaN
9,hvOWbFacsVZwFIAAXtN3BA,cheekwood-nashville,Cheekwood,https://s3-media1.fl.yelpcdn.com/bphoto/pZCkb4...,False,https://www.yelp.com/biz/cheekwood-nashville?a...,295,"[{'alias': 'gardens', 'title': 'Botanical Gard...",4.0,"{'latitude': 36.0881153743368, 'longitude': -8...",[],"{'address1': '1200 Forrest Park Dr', 'address2...",+16153568000,(615) 356-8000,13670.616212,NaN


In [17]:
yelp_api = YelpAPI(api_key)
response4 = yelp_api.search_query(term = 'park', location = 'Cheatham County, TN', search_limit = 200)
response4

{'businesses': [{'id': 'e3YMcMeIAg0zzEF2-hbPYQ',
   'alias': 'harpeth-river-state-park-kingston-springs-2',
   'name': 'Harpeth River State Park',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0TEPOQME6AE-_LSVA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/harpeth-river-state-park-kingston-springs-2?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 16,
   'categories': [{'alias': 'parks', 'title': 'Parks'},
    {'alias': 'hiking', 'title': 'Hiking'},
    {'alias': 'fishing', 'title': 'Fishing'}],
   'rating': 4.0,
   'coordinates': {'latitude': 36.15071045450449,
    'longitude': -87.11617411765306},
   'transactions': [],
   'location': {'address1': '1640 Cedar Hill Rd',
    'address2': '',
    'address3': '',
    'city': 'Kingston Springs',
    'zip_code': '37082',
    'country': 'US',
    'state': 'TN',
    'display_address': ['1640 Cedar Hill R

In [18]:
cols = list(response4['businesses'][0].keys())
data4 = pd.DataFrame(columns=cols)

for biz in response4['businesses']:
    data4 = data4.append(biz, ignore_index=True)
                    
data4.head(50)   #has 20 parks no matter limit is 100 or 200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance
0,e3YMcMeIAg0zzEF2-hbPYQ,harpeth-river-state-park-kingston-springs-2,Harpeth River State Park,https://s3-media3.fl.yelpcdn.com/bphoto/D_Ctr0...,False,https://www.yelp.com/biz/harpeth-river-state-p...,16,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.15071045450449, 'longitude': -...",[],"{'address1': '1640 Cedar Hill Rd', 'address2':...",+16159522099,(615) 952-2099,10955.663709
1,u7_bvseJeO8dB9nJcbbXhQ,sycamore-creek-recreational-area-ashland-city,Sycamore Creek Recreational Area,https://s3-media3.fl.yelpcdn.com/bphoto/jAsCZO...,False,https://www.yelp.com/biz/sycamore-creek-recrea...,2,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",3.5,"{'latitude': 36.2713508605957, 'longitude': -8...",[],"{'address1': '', 'address2': '', 'address3': N...",,,4134.589736
2,7s-bi0Z-OrIDJmfSfrrDOg,beaman-park-nature-center-nashville-2,Beaman Park Nature Center,https://s3-media3.fl.yelpcdn.com/bphoto/XJuSpJ...,False,https://www.yelp.com/biz/beaman-park-nature-ce...,26,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.262533, 'longitude': -86.894313}",[],"{'address1': '5911 Old Hickory Blvd', 'address...",+16158628580,(615) 862-8580,18233.689678
3,ktsVFphtejDS-knReeMu-A,hidden-lake-state-park-nashville,Hidden Lake State Park,https://s3-media4.fl.yelpcdn.com/bphoto/sY9pmw...,False,https://www.yelp.com/biz/hidden-lake-state-par...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.0926605944158, 'longitude': -8...",[],"{'address1': '7851 McCrory Ln', 'address2': ''...",,,18345.347456
4,docjbpsuMVzYyF-8TVEEBQ,rockland-recreation-area-hendersonville,Rockland Recreation Area,https://s3-media1.fl.yelpcdn.com/bphoto/7ePq3z...,False,https://www.yelp.com/biz/rockland-recreation-a...,4,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.3009417057037, 'longitude': -8...",[],"{'address1': '5 Power Plant Rd', 'address2': '...",+16158224846,(615) 822-4846,40767.341539
5,5-uPWHnIvFJ-p0FSVB3iMw,balthrop-park-pleasant-view,Balthrop Park,https://s3-media3.fl.yelpcdn.com/bphoto/yLGZmU...,False,https://www.yelp.com/biz/balthrop-park-pleasan...,1,"[{'alias': 'parks', 'title': 'Parks'}]",4.0,"{'latitude': 36.397113, 'longitude': -87.03415}",[],"{'address1': '2622 Church St', 'address2': '',...",+16157460600,(615) 746-0600,17506.947161
6,L093rN4oadwNmCyiPAEfEQ,franklin-simpson-park-and-recreation-franklin,Franklin Simpson Park and Recreation,https://s3-media3.fl.yelpcdn.com/bphoto/7bw5YK...,False,https://www.yelp.com/biz/franklin-simpson-park...,1,"[{'alias': 'parks', 'title': 'Parks'}]",5.0,"{'latitude': 36.7240486, 'longitude': -86.5525...",[],"{'address1': 'Filter Plant Rd', 'address2': ''...",+12705868999,(270) 586-8999,71870.347511
7,8ypQ14bfksJtA7hL2nEmcw,montgomery-bell-state-park-burns,Montgomery Bell State Park,https://s3-media4.fl.yelpcdn.com/bphoto/FBF26o...,False,https://www.yelp.com/biz/montgomery-bell-state...,46,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.095961, 'longitude': -87.286169}",[],"{'address1': '1020 Jackson Hill Rd', 'address2...",+16157979052,(615) 797-9052,23969.643356
8,vcB9gExfizEWWrx5N-4JJA,harpeth-river-greenway-nashville,Harpeth River Greenway,https://s3-media3.fl.yelpcdn.com/bphoto/D-GnJb...,False,https://www.yelp.com/biz/harpeth-river-greenwa...,6,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.07738, 'longitude': -86.95922}",[],"{'address1': '7820 Coley Davis Rd', 'address2'...",,,22714.081030
9,njenDpIerIT38E4lgIhjkA,gregory-mill-dam-park-smyrna,Gregory Mill Dam Park,https://s3-media2.fl.yelpcdn.com/bphoto/4v2vjK...,False,https://www.yelp.com/biz/gregory-mill-dam-park...,4,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 35.9739189, 'longitude': -86.5039...",[],"{'address1': '390 Enon Springs Rd E', 'address...",+16154599773,(615) 459-9773,61343.428654


In [19]:
yelp_api = YelpAPI(api_key)
response5 = yelp_api.search_query(term = 'park', location = 'Nashville County, TN', search_limit = 500)
response5

{'businesses': [{'id': 'zzXDi0Pdv0s84M-oQaIa_g',
   'alias': 'radnor-lake-state-park-nashville',
   'name': 'Radnor Lake State Park',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/x_k73AlKQx0-cjGwXKtuHQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/radnor-lake-state-park-nashville?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 208,
   'categories': [{'alias': 'parks', 'title': 'Parks'}],
   'rating': 4.5,
   'coordinates': {'latitude': 36.062764, 'longitude': -86.809891},
   'transactions': [],
   'location': {'address1': '1160 Otter Creek Rd',
    'address2': '',
    'address3': '',
    'city': 'Nashville',
    'zip_code': '37220',
    'country': 'US',
    'state': 'TN',
    'display_address': ['1160 Otter Creek Rd', 'Nashville, TN 37220']},
   'phone': '+16153733467',
   'display_phone': '(615) 373-3467',
   'distance': 8886.359114102528},
  {'id':

In [20]:
cols = list(response5['businesses'][0].keys())
data5 = pd.DataFrame(columns=cols)

for biz in response5['businesses']:
    data5 = data5.append(biz, ignore_index=True)
                    
data5.head(50)   #has 20 parks no matter limit is 100 or 200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance
0,zzXDi0Pdv0s84M-oQaIa_g,radnor-lake-state-park-nashville,Radnor Lake State Park,https://s3-media1.fl.yelpcdn.com/bphoto/x_k73A...,False,https://www.yelp.com/biz/radnor-lake-state-par...,208,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.062764, 'longitude': -86.809891}",[],"{'address1': '1160 Otter Creek Rd', 'address2'...",+16153733467,(615) 373-3467,8886.359114
1,WjWDl782jABdLzUu9nof3g,shelby-park-nashville,Shelby Park,https://s3-media2.fl.yelpcdn.com/bphoto/hJoCFP...,False,https://www.yelp.com/biz/shelby-park-nashville...,56,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.173435340130204, 'longitude': ...",[],"{'address1': '2021 Fatherland St', 'address2':...",+16158628474,(615) 862-8474,5446.490507
2,CRmZDx9rW3HFo0Z9FbfyEw,cumberland-park-nashville,Cumberland Park,https://s3-media1.fl.yelpcdn.com/bphoto/xRQAmw...,False,https://www.yelp.com/biz/cumberland-park-nashv...,32,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.1626862381255, 'longitude': -8...",[],"{'address1': '592 S 1st St', 'address2': '', '...",,,2811.838069
3,73esU0u3hYIPPqCScAA2HQ,centennial-park-nashville,Centennial Park,https://s3-media3.fl.yelpcdn.com/bphoto/yiE3AL...,False,https://www.yelp.com/biz/centennial-park-nashv...,219,"[{'alias': 'parks', 'title': 'Parks'}]",4.5,"{'latitude': 36.149064, 'longitude': -86.811993}",[],"{'address1': '2500 W End Ave', 'address2': '',...",,,3511.073612
4,Wjv13Wi_9RPRJPg-uMkcRQ,bicentennial-capitol-mall-state-park-nashville-2,Bicentennial Capitol Mall State Park,https://s3-media4.fl.yelpcdn.com/bphoto/I5sZVJ...,False,https://www.yelp.com/biz/bicentennial-capitol-...,86,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.1680549, 'longitude': -86.7863...",[],"{'address1': '600 James Robertson Pkwy', 'addr...",+16157415280,(615) 741-5280,3512.519021
5,yu9e-_S7U97kleJQw0eOzQ,richland-creek-greenway-nashville,Richland Creek Greenway,https://s3-media4.fl.yelpcdn.com/bphoto/fiqTzB...,False,https://www.yelp.com/biz/richland-creek-greenw...,30,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.5,"{'latitude': 36.140481933298005, 'longitude': ...",[],"{'address1': '4601 Murphy Rd', 'address2': '',...",,,6084.633102
6,DiG2uoaL8ZLQcpGlkHraag,long-hunter-state-park-hermitage,Long Hunter State Park,https://s3-media1.fl.yelpcdn.com/bphoto/KLoy5s...,False,https://www.yelp.com/biz/long-hunter-state-par...,38,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.0943884, 'longitude': -86.5571...",[],"{'address1': '2910 Hobson Pike', 'address2': '...",+16158852422,(615) 885-2422,20157.552179
7,eQB8iaJ5utCWnm4P2qUcCw,percy-priest-lake-nashville,Percy Priest Lake,https://s3-media2.fl.yelpcdn.com/bphoto/abQBy2...,False,https://www.yelp.com/biz/percy-priest-lake-nas...,14,"[{'alias': 'lakes', 'title': 'Lakes'}, {'alias...",4.5,"{'latitude': 36.156946, 'longitude': -86.6175}",[],"{'address1': '3737 Bell Rd', 'address2': '', '...",+16158891975,(615) 889-1975,14348.497419
8,356dzXl7YXrWq6MrvSz23Q,sevier-park-nashville,Sevier Park,https://s3-media3.fl.yelpcdn.com/bphoto/HBiJWi...,False,https://www.yelp.com/biz/sevier-park-nashville...,39,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.5,"{'latitude': 36.1195273, 'longitude': -86.7907...",[],"{'address1': '3000 Granny White Pike', 'addres...",,,2381.754220
9,GoLUrmNHkh1x__YUUr21RQ,two-rivers-park-nashville,Two Rivers Park,https://s3-media1.fl.yelpcdn.com/bphoto/YMMGc5...,False,https://www.yelp.com/biz/two-rivers-park-nashv...,15,"[{'alias': 'parks', 'title': 'Parks'}, {'alias...",4.0,"{'latitude': 36.185593, 'longitude': -86.678432}",[],"{'address1': '3150 Mcgavock Pike', 'address2':...",+16158837781,(615) 883-7781,10213.656019


In [21]:
yelp_api = YelpAPI(api_key)
response6 = yelp_api.search_query(term = 'family friendly', location = 'Wilson County, TN', search_limit = 200)
response6

{'businesses': [{'id': 'z_sXeyRnGjmbmJagS-PxFg',
   'alias': 'town-square-social-lebanon',
   'name': 'Town Square Social',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/Li-geBecv9qd4_8YAXPSfg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/town-square-social-lebanon?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 191,
   'categories': [{'alias': 'newamerican', 'title': 'American (New)'}],
   'rating': 4.5,
   'coordinates': {'latitude': 36.2075998381492,
    'longitude': -86.2906668358173},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '145 Public Square',
    'address2': '',
    'address3': None,
    'city': 'Lebanon',
    'zip_code': '37087',
    'country': 'US',
    'state': 'TN',
    'display_address': ['145 Public Square', 'Lebanon, TN 37087']},
   'phone': '+16157844593',
   'display_phone': '(615) 784-4593',
   'distance':

In [22]:
cols = list(response6['businesses'][0].keys())
data6 = pd.DataFrame(columns=cols)

for biz in response6['businesses']:
    data6 = data6.append(biz, ignore_index=True)
                    
data6.head(50)   #has 20 parks no matter limit is 100 or 200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,z_sXeyRnGjmbmJagS-PxFg,town-square-social-lebanon,Town Square Social,https://s3-media4.fl.yelpcdn.com/bphoto/Li-geB...,False,https://www.yelp.com/biz/town-square-social-le...,191,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 36.2075998381492, 'longitude': -8...",[],$$,"{'address1': '145 Public Square', 'address2': ...",+16157844593,(615) 784-4593,6014.923440
1,Lh7_jvQ3axujF7148Dh-6A,samis-brick-oven-pizzeria-murfreesboro-4,Sami's Brick Oven Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/9VJW5h...,False,https://www.yelp.com/biz/samis-brick-oven-pizz...,129,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.84226, 'longitude': -86.39053}","[delivery, pickup]",$,"{'address1': '129 SE Broad St', 'address2': No...",+16154745411,(615) 474-5411,35640.236207
2,fuw8VRMaPEaUkQyU4_IETg,huckleberry-brewing-franklin,Huckleberry Brewing,https://s3-media3.fl.yelpcdn.com/bphoto/6liqM6...,False,https://www.yelp.com/biz/huckleberry-brewing-f...,12,"[{'alias': 'brewpubs', 'title': 'Brewpubs'}]",4.0,"{'latitude': 35.94488, 'longitude': -86.82132}",[],NaN,"{'address1': '600A Frazier Dr', 'address2': No...",+16157217924,(615) 721-7924,51884.038202
3,YOmh25YRJSDuGqt7zjEHEA,gondola-house-pizzeria-hermitage,Gondola House Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/jLc3Q4...,False,https://www.yelp.com/biz/gondola-house-pizzeri...,279,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 36.214619, 'longitude': -86.59642}","[delivery, pickup]",$$,"{'address1': '4613 Lebanon Pike', 'address2': ...",+16156799033,(615) 679-9033,26913.866560
4,O7gTnoBLdPpzAySbE9iAgA,simple-grill-smyrna,Simple Grill,https://s3-media1.fl.yelpcdn.com/bphoto/DoGd8a...,False,https://www.yelp.com/biz/simple-grill-smyrna?a...,1,"[{'alias': 'tradamerican', 'title': 'American ...",5.0,"{'latitude': 35.98396630000001, 'longitude': -...",[],NaN,"{'address1': '901 Rock Springs Rd', 'address2'...",+16159844053,(615) 984-4053,28832.215926
5,peoo15_zdEuEf6EVJA09kA,sorelles-eagleville,Sorelles,https://s3-media3.fl.yelpcdn.com/bphoto/8kNMQm...,False,https://www.yelp.com/biz/sorelles-eagleville?a...,8,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 35.74262176092883, 'longitude': -...",[],NaN,"{'address1': '161 N Main St', 'address2': '', ...",+16156400707,(615) 640-0707,55344.703258
6,OWOOc0YjU_kioLeEgo5VCA,the-loveless-cafe-nashville,The Loveless Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/78UVcr...,False,https://www.yelp.com/biz/the-loveless-cafe-nas...,2531,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 36.035175, 'longitude': -86.972013}","[delivery, pickup]",$$,"{'address1': '8400 Hwy 100', 'address2': '', '...",+16156469700,(615) 646-9700,61317.746093
7,FmxYcT3LWjs4FX41VQjl1w,m-l-rose-craft-beer-and-burgers-mt-juliet,M L Rose Craft Beer & Burgers,https://s3-media1.fl.yelpcdn.com/bphoto/TtwXlW...,False,https://www.yelp.com/biz/m-l-rose-craft-beer-a...,53,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 36.16653, 'longitude': -86.51374}",[pickup],$$,"{'address1': '401 S Mt Juliet Rd', 'address2':...",+16152888269,(615) 288-8269,18493.855536
8,PSCaqN9wjkngJsCyXGm7Gg,valley-home-farm-wartrace,Valley Home Farm,https://s3-media3.fl.yelpcdn.com/bphoto/dTo4mX...,False,https://www.yelp.com/biz/valley-home-farm-wart...,7,"[{'alias': 'pickyourown', 'title': 'Pick Your ...",5.0,"{'latitude': 35.53043, 'longitude': -86.31296}",[],NaN,"{'address1': '310 Potts Rd', 'address2': '', '...",+19313896470,(931) 389-6470,69450.787174
9,MS9cjY1kC4c73sVjJ_53WQ,old-school-pizza-brentwood,Old School Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/MesVoA...,False,https://www.yelp.com/biz/old-school-pizza-bren...,11,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 36.03154, 'longitude': -86.80285}","[delivery, pickup]",NaN,"{'addres

In [23]:
yelp_api = YelpAPI(api_key)
response = yelp_api.search_query(term = 'museum', location = 'Wilson County, TN', search_limit = 200)
response

{'businesses': [{'id': 'mEGMDswcZqoAYda_bLZMdw',
   'alias': 'days-gone-by-museum-portland',
   'name': 'Days Gone By Museum',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/fHh_aqaiW2I7lCVf7mxDgw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/days-gone-by-museum-portland?adjust_creative=MQNwvLri_UXL3Jrl-lPNmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=MQNwvLri_UXL3Jrl-lPNmg',
   'review_count': 1,
   'categories': [{'alias': 'museums', 'title': 'Museums'}],
   'rating': 5.0,
   'coordinates': {'latitude': 36.591769605875,
    'longitude': -86.5179636329412},
   'transactions': [],
   'location': {'address1': '122 Davis St',
    'address2': '',
    'address3': '',
    'city': 'Portland',
    'zip_code': '37148',
    'country': 'US',
    'state': 'TN',
    'display_address': ['122 Davis St', 'Portland, TN 37148']},
   'phone': '+16153252555',
   'display_phone': '(615) 325-2555',
   'distance': 52290.695260338754},
  {'id': 'mXqcL

In [24]:
cols = list(response['businesses'][0].keys())
data = pd.DataFrame(columns=cols)

for biz in response['businesses']:
    data = data.append(biz, ignore_index=True)
                    
data.head(50)   #has 20 parks no matter limit is 100 or 200

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,mEGMDswcZqoAYda_bLZMdw,days-gone-by-museum-portland,Days Gone By Museum,https://s3-media1.fl.yelpcdn.com/bphoto/fHh_aq...,False,https://www.yelp.com/biz/days-gone-by-museum-p...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.591769605875, 'longitude': -86...",[],"{'address1': '122 Davis St', 'address2': '', '...",+16153252555,(615) 325-2555,52290.695260,NaN
1,mXqcL-AQDLDXiT_dsksXGQ,belmont-mansion-nashville,Belmont Mansion,https://s3-media2.fl.yelpcdn.com/bphoto/11vctE...,False,https://www.yelp.com/biz/belmont-mansion-nashv...,61,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.5,"{'latitude': 36.13544467, 'longitude': -86.795...",[],"{'address1': '1700 Acklen Ave', 'address2': ''...",+16154605459,(615) 460-5459,43964.796045,NaN
2,292h9a8i53ggVZvWy_g6jA,wilson-county-veterans-museum-lebanon,Wilson County Veterans Museum,https://s3-media4.fl.yelpcdn.com/bphoto/2bNzxZ...,False,https://www.yelp.com/biz/wilson-county-veteran...,1,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.20703, 'longitude': -86.28736}",[],"{'address1': '228 E Main St', 'address2': '', ...",+16154442460,(615) 444-2460,6025.653003,NaN
3,kcQa-hcsWQ8VyQfaQFVo8A,cragfont-state-historic-site-castalian-springs,Cragfont State Historic Site,https://s3-media3.fl.yelpcdn.com/bphoto/kEc8y5...,False,https://www.yelp.com/biz/cragfont-state-histor...,4,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.404644, 'longitude': -86.342149}",[],"{'address1': '200 Cragfont Rd', 'address2': ''...",+16154527070,(615) 452-7070,27958.879692,NaN
4,VIKQVpBQfrHP7cs8ZZo8Og,earth-experience-murfreesboro,Earth Experience,https://s3-media2.fl.yelpcdn.com/bphoto/MB4Csk...,False,https://www.yelp.com/biz/earth-experience-murf...,13,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 35.8370535324916, 'longitude': -8...",[],"{'address1': '816 Old Salem Rd', 'address2': '...",+16159008358,(615) 900-8358,36395.536272,NaN
5,rryEC-u2GnR0Z2_l_bUXtA,andrew-jacksons-hermitage-nashville-2,Andrew Jackson's Hermitage,https://s3-media4.fl.yelpcdn.com/bphoto/Mnsz3I...,False,https://www.yelp.com/biz/andrew-jacksons-hermi...,300,"[{'alias': 'museums', 'title': 'Museums'}, {'a...",4.5,"{'latitude': 36.2150124, 'longitude': -86.6129...",[],"{'address1': '4580 Rachel's Ln', 'address2': '...",+16158892941,(615) 889-2941,28360.117529,NaN
6,mnwTbHyIgpMvOG9705zqWA,country-music-hall-of-fame-and-museum-nashville-3,Country Music Hall of Fame and Museum,https://s3-media3.fl.yelpcdn.com/bphoto/lr2eAA...,False,https://www.yelp.com/biz/country-music-hall-of...,687,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.158266, 'longitude': -86.776126}",[],"{'address1': '222 Rep. John Lewis Way S', 'add...",+16154162001,(615) 416-2001,42223.015690,NaN
7,h76_04i00MtqxosHBHWS9g,cookeville-history-museum-cookeville,Cookeville History Museum,https://s3-media2.fl.yelpcdn.com/bphoto/yH6Oik...,False,https://www.yelp.com/biz/cookeville-history-mu...,1,"[{'alias': 'museums', 'title': 'Museums'}]",5.0,"{'latitude': 36.1635399, 'longitude': -85.5048...",[],"{'address1': '40 E Broad St', 'address2': None...",+19315205455,(931) 520-5455,71918.480174,NaN
8,aFsDsPcsac8ZskCn20WuDg,game-galaxy-smyrna,Game Galaxy,https://s3-media1.fl.yelpcdn.com/bphoto/-YQPSz...,False,https://www.yelp.com/biz/game-galaxy-smyrna?ad...,23,"[{'alias': 'arcades', 'title': 'Arcades'}]",4.5,"{'latitude': 35.9829368591309, 'longitude': -8...",[],"{'address1': '3 S Lowry St', 'address2': '', '...",+16157517660,(615) 751-7660,27052.324308,NaN
9,jSYu2apG1uYq3eNzSA-OHw,johnny-cash-museum-nashville-4,Johnny Cash Museum,https://s3-media2.fl.yelpcdn.com/bphoto/qkanvY...,False,https://www.yelp.com/biz/johnny-cash-museum-na...,431,"[{'alias': 'museums', 'title': 'Museums'}]",4.0,"{'latitude': 36.16084, 'longitude': -86.77586}",[],"{'address1': '119 3rd Av

In [ ]:
yelp_api = YelpAPI(api_key)
response = yelp_api.search_query(term = 'park', location = 'Wilson County, TN', search_limit = 200)
response

In [ ]:
yelp_api = YelpAPI(api_key)
response = yelp_api.search_query(term = 'park', location = 'Wilson County, TN', search_limit = 200)
response

In [ ]:
yelp_api = YelpAPI(api_key)
response = yelp_api.search_query(term = 'park', location = 'Wilson County, TN', search_limit = 200)
response